# Домашнее задание 2

## Задание 1: Байесовская классификация

#### Открыть в Pandas файл **names.csv** (см. вложения). Ответить на вопросы ниже, используя средства языка Python и необходимых библиотек

In [1]:
#imports
import numpy as np
import pandas as pd
# import sklearn
import matplotlib.pyplot as plt
# import scipy
from IPython.display import Math
from sympy import latex, Matrix

In [2]:
csv_path = '/home/echo/Documents/HSE/ML/names.csv'
df = pd.read_csv(csv_path, names=['Name', 'Sex']) # dataframe
display(df.head(), df.shape, df['Sex'].unique())
display(df[df['Sex'].str.contains('boy')].shape[0])

,Name,Sex
0,John,boy
1,William,boy
2,James,boy
3,Charles,boy
4,George,boy


(6782, 2)

array(['boy', 'girl'], dtype=object)

3437

In [3]:
# let's transform our data
# df_new = pd.DataFrame(df, columns=['Name', 'boy'])
df['Sex'] = (df['Sex'] == 'boy').astype(int)
df.head()
display(df.head(), df['Sex'].sum())

,Name,Sex
0,John,1
1,William,1
2,James,1
3,Charles,1
4,George,1


3437

In [4]:
# for label in ['Name']:
# #     print(label)
#     plt.hist(df[df["Sex"] == 1][label], color='blue', label='boy', alpha=0.7, density=True)
#     plt.hist(df[df["Sex"] == 0][label], color='red', label='girl', alpha=0.7, density=True)
#     plt.title(label)
#     plt.ylabel("Probability")
#     plt.xlabel(label)
#     plt.legend()
#     plt.show()


#### Разделить данные в выборке на обучающий набор и тестирование (выбор принципа разделения за вами – например, 70% данных в обучении и 30% в тестировании)

In [4]:
train_set, valid, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])
# df.sample return random sample of data (shuffle)
# 0-60% -> train, 60-80% -> valid, 80-100% test
display(len(train_set)/len(df), len(valid)/len(df), len(test)/len(df))


0.59997051017399

0.19994102034797995

0.20008846947803008

# I.S's functions
from collections import defaultdict
from math import log


def train(samples):
    classes, freq = defaultdict(lambda:0), defaultdict(lambda:0) # 0 is default value
    for feats, label in samples:
        classes[label] += 1                 # count classes frequencies
        for feat in feats:
            freq[label, feat] += 1          # count features frequencies
#     display(freq)
    for label, feat in freq:                # normalize features frequencies
        freq[label, feat] /= classes[label]
    for c in classes:                       # normalize classes frequencies
        classes[c] /= len(samples)
#     print(classes, freq)
#     print('classes:', dict(classes))
#     print(dict(freq))
    return classes, freq                    # return P(C) and P(O|C)
# P(O|C) = P(C|O) * P(O) / P(C)

def MyLog(x):
    if x > 0:
        return log(x)
    else:
        return 0
    
def classify_legacy(classifier, feats):
    classes, prob = classifier # P(C) и P(O|C)
    # мы выбираем такой класс, который максимизирует вероятность (через знания - обратную вероятность)
   # calculate argmin(-log(C|O))
    return min(classes.keys(),  # 
        key = lambda cl: -MyLog(classes[cl]) + \
            sum(-MyLog(prob.get((cl,feat), 0)) for feat in feats))
    # по ключу будут вычислятся значения для сравнения
    # ФОРМУЛА ВЫШЕ БЫЛА ПОЛОМАНА

# argmax = lambda c_k: 
# def classify_legacy(classifier, feats):
#     classes, prob = classifier # P(C) и P(O|C)
#     return min(classes.keys(),
#         key = lambda cl: -MyLog(classes[cl]) + \
#             sum(-MyLog(prob.get((cl,feat), 0)) for feat in feats))

   
def get_features(sample): return (sample[-1], sample[0]) # get last letter and the first



In [9]:
# Training
features_train = [(get_features(sample.Name), sample.Sex) for sample in train_set.iloc]
classifier = train(features_train)

In [17]:
# display(test.head())

# sample is normalized
# print(len(test[test["Sex"] == 0]))
# print(len(test[test["Sex"] == 1]))
sample_size = len(test)
errors = 0

features_test = [(get_features(sample.Name), sample.Sex) for sample in test.iloc]

for feat, label in features_test:
    ans = classify_legacy(classifier, feat)
    if ans != label:
        errors += 1
    
# print(f'Accuracy = {(sample_size-errors)/sample_size}')
display(Math(f'Accuracy = {(sample_size-errors)/sample_size}'))
    

<IPython.core.display.Math object>

In [11]:
# display(classifier)
# classes, prob = classifier
# display(classes, prob)
# # display(prob.get())
# display(classes.keys())
# # print('--', -log(classes[cl]))
# display(min(classes.keys()),
# key = lambda cl: -log(classes[cl]) + sum(-log(prob.get((cl,feat))) for feat in feats))

In [ ]:
# print('gender: ', classify(classifier, ('a', 'M') ))
# a =get_features(u'Milana')

#### Обучить наивную байесовскую классификацию из файла **Sem2.ipynb** (см. вложения) на тренировочном наборе данных. Затем с помощью метода **classify()** разметить имена по полу в тестировочном наборе данных

### Метрики

In [14]:
from sklearn.metrics import precision_recall_curve, classification_report

In [33]:
def get_data(df):
    features_test = [(get_features(sample.Name), sample.Sex) for sample in df.iloc]
    x_test = np.empty(len(features_test), dtype=object)
    y_test = np.zeros(len(features_test))
    i = 0
    for feat, label in features_test:
        x_test[i] = feat
        y_test[i] = label
        i += 1
    return x_test, y_test

In [60]:
x_test, y_test = get_data(test)
print(len(x_test))
def classify_data(features):
    return np.array([classify_legacy(classifier, sample) for sample in features])
# y_pred = np.array([classify_legacy(classifier, sample) for sample in x_test])
y_pred = classify_data(x_test)
for y, y_hat in zip(y_test, y_pred):
    print(y, y_hat)
# x_test, y_test = features_test

1357
1.0 1
0.0 0
1.0 0
1.0 1
0.0 0
1.0 1
1.0 1
1.0 1
1.0 1
0.0 1
0.0 1
0.0 0
0.0 1
1.0 1
0.0 0
0.0 0
1.0 0
0.0 0
0.0 1
0.0 0
0.0 0
1.0 1
1.0 1
0.0 0
0.0 0
0.0 1
0.0 0
0.0 0
1.0 1
0.0 0
0.0 1
0.0 0
1.0 1
1.0 1
1.0 1
1.0 1
0.0 0
0.0 0
1.0 1
1.0 0
1.0 1
0.0 0
1.0 1
0.0 1
1.0 1
0.0 0
0.0 0
1.0 1
0.0 1
0.0 0
0.0 0
0.0 0
1.0 1
0.0 0
1.0 0
1.0 0
0.0 0
1.0 1
0.0 1
1.0 1
0.0 0
0.0 0
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
0.0 0
0.0 0
0.0 0
0.0 0
0.0 1
1.0 1
1.0 1
1.0 1
1.0 1
1.0 1
0.0 0
0.0 0
0.0 0
1.0 1
1.0 1
0.0 0
0.0 0
1.0 0
1.0 1
0.0 0
1.0 1
1.0 1
1.0 1
0.0 1
0.0 0
1.0 1
0.0 0
0.0 1
0.0 1
1.0 1
0.0 0
1.0 1
0.0 1
1.0 1
1.0 1
0.0 0
1.0 1
1.0 1
1.0 1
0.0 0
1.0 1
1.0 1
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
1.0 1
1.0 1
1.0 0
0.0 0
0.0 1
0.0 0
0.0 1
0.0 0
0.0 1
1.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
1.0 1
1.0 1
1.0 1
0.0 0
1.0 1
0.0 0
1.0 1
1.0 1
0.0 0
0.0 0
1.0 1
0.0 0
1.0 0
1.0 1
0.0 0
1.0 1
1.0 1
0.0 0
0.0 0
1.0 1
1.0 1
0.0 0
0.0 0
0.0 0
1.0 1
1.0 1
0.0 0
0.0 1
0.0 0
0.0 0
0.0 0
0.0 0
1.0 1
1.0 1
0.0 0

In [62]:
report = classification_report(y_test, y_pred, target_names=['girl', 'boy'])
print(report)

              precision    recall  f1-score   support

        girl       0.79      0.77      0.78       677
         boy       0.77      0.80      0.79       680

    accuracy                           0.78      1357
   macro avg       0.78      0.78      0.78      1357
weighted avg       0.78      0.78      0.78      1357

